
# Hyperparameter Optimization Using Keras Tuner API

[Hyperparameter optimization](https://www.jeremyjordan.me/hyperparameter-tuning/) is important if you're trying to make a model state-of-the-art. For instance, if you're developing a new architecture for image classification, you'll like to set such a value for the number of output units ( output dimensionality ) which would give you convergence quickly during training. Accuracy also depends hugely on hyperparameters ( like batch-size, learning rate, weight decay ).

We'll be using the [Keras Tuner API](https://keras-team.github.io/keras-tuner/) which hyperparameter optimization right in our TF Keras models and that too in an easy-to-use interface.

You may read the [TensorFlow Blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html) and [official tutorial](https://www.tensorflow.org/tutorials/keras/keras_tuner#overview).



## 1) Import TensorFlow and install kerastuner via pip

We'll install `kerastuner` package using pip.


In [ ]:

from tensorflow import keras
import tensorflow as tf
import IPython

!pip install keras-tuner



## 2) Preparing the CIFAR10 dataset for training the model

We'll require some data to train and optimize our model. We use the [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset as it's readily available in `tf.keras.datasets` module.
Also, we normalize the pixel values between 0 and 1.


In [ ]:

( train_img, train_label ), ( test_img , test_label ) = keras.datasets.cifar10.load_data()

train_img = train_img.astype( 'float32' ) / 255
test_img = test_img.astype( 'float32' ) / 255

train_label = keras.utils.to_categorical( train_label , num_classes=10 ) 
test_label = keras.utils.to_categorical( test_label , num_classes=10 ) 



## 3) Define a CNN model with hyperparameters

We'll use a Convolutional Neural Network to classify images of the CIFAR10 dataset. We'll use two Conv2D layers and optimize their `units` argument. This method creates a new model with the given hyperparameter ( `hp` ).


In [ ]:

import kerastuner as kt

def model( hp ):
    model = keras.Sequential()
    hp_units = hp.Int( 'units', min_value=32, max_value=512, step=32)
    model.add( keras.layers.Conv2D( hp_units , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' , input_shape=( 32 , 32 , 3 ) ) )
    model.add( keras.layers.Conv2D( hp_units , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' , input_shape=( 32 , 32 , 3 ) ) )
    model.add( keras.layers.Flatten() )
    tf.keras.activations.relu
    activation = hp.Choice( 'activation' , values=[ 'selu' , 'relu' , 'leaky_relu' ] )
    model.add( keras.layers.Activation( activation ) )
    model.add( keras.layers.Dense( 10 , activation='softmax' ) )
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss='categorical_crossentropy', 
                metrics = ['accuracy'])
    return model



Intialize the `Tuner`. We'll use [Hyperband algorithm](https://arxiv.org/pdf/1603.06560.pdf) for optimization.


In [ ]:

tuner = kt.tuners.Hyperband( model,
                     objective='val_accuracy', 
                     max_epochs=10,
                     factor=3,
                     directory='opt_dir',
                     project_name='cifar10CNNopt' )      



## 4) Running the `tuner` for optimization

We supply the training and testing data to the tuner for hyperparamter optimization.


In [ ]:

class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

tuner.search( train_img , train_label , epochs=15 , validation_data=( test_img , test_label ), callbacks = [ClearTrainingOutput()])

# Optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")



Once the optimization is done, take the best parameters and load our model with them.


In [ ]:

# Load the optimal parameters
model = tuner.hypermodel.build(best_hps)

# Train the model using them
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))



That's All! Keras just became more great!
